In [156]:
import random
import numpy  as np
import pandas as pd
from scipy.spatial import distance_matrix
from ortools.graph import pywrapgraph
from sklearn.neighbors         import KNeighborsRegressor
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

## Data Wrangling

Extracting the data from the data file.

In [157]:
#Returns a list of integer lists.
def list_lines(file_name):
    with open(file_name) as file:
        lines = file.read().splitlines()
    line_list = [[int(n) for  n in ll.split()] for ll in lines]
    return line_list

#Provides the dividing line between warehouse and order sections in the line list.
def find_wh_lines(line_list):
    wh_count = line_list[3][0]
    wh_endline = (wh_count*2)+4
    return wh_endline

#Returns a 2-d array of products(rows) by warehouses(columns).
def get_inventories(line_list):
    wh_endline = find_wh_lines(line_list)
    invs = line_list[5:wh_endline+1:2]
    supply = np.array(invs).transpose()
    return supply.astype(np.int16)

#Returns a 2-d array of products(rows) by orders(columns).
def get_orders(line_list):
    wh_endline = find_wh_lines(line_list)
    demand = np.zeros((line_list[1][0], line_list[wh_endline][0]),dtype=np.int16)
    orders = line_list[wh_endline+3::3]
    for i,ord in enumerate(orders):
        for prod in ord:
            demand[prod, i] += 1
    return demand.astype(np.int16)

#Returns the locations of Warehouses and the Orders.
def get_locs(line_list):
    wh_endline = find_wh_lines(line_list)
    wh_locs = np.array(line_list[4:wh_endline:2])
    cust_locs = np.array(line_list[wh_endline+1::3])
    return wh_locs.astype(np.int16), cust_locs.astype(np.int16)

In [158]:
line_list = list_lines('data.txt')
supply = get_inventories(line_list) #Item vs Qty present in each warehouse (shape: 400x10)
demand = get_orders(line_list) #Item vs Qty required in each order (shape: 400x1250)
warehouse_loctions, order_loctions = get_locs(line_list) #Location of warehouse and orders
distances_mtx = distance_matrix(order_loctions, warehouse_loctions) # Order vs Warehouse Distance (shape: 1250x10)

In [ ]:
#rough work
assign_df   = pd.DataFrame([2,2, 3, 5,5,5,5,5, 1,1,1, 6,6,6,6, 4,4,4], columns=['order'])
penalize_df = pd.DataFrame([6,5,1,1,3,5,6,5], columns=['order'])

assign_group   = assign_df.groupby('order')
penalize_group = penalize_df.groupby('order')
assign_group_count   = assign_group['order'].count()
penalize_group_count = penalize_group['order'].count()
zero_series = pd.Series(np.repeat(0,10))

padded_assign_group_count   = (zero_series + assign_group_count).fillna(1)
padded_penalize_group_count = (zero_series + penalize_group_count).fillna(0)

print("penal")
print(padded_penalize_group_count)
print("assign")
print(padded_assign_group_count)
ratio = padded_penalize_group_count.divide(padded_assign_group_count).tolist()
# print("---")
print(ratio)
threshold_exceed_bool  = pd.Series(ratio) > 0.4
print(threshold_exceed_bool)
ratio_df     = pd.DataFrame(ratio, columns=['ratio'])
new_ratio_df = pd.DataFrame(np.repeat(1.0,10), columns=['ratio'])
new_ratio_df[threshold_exceed_bool] += ratio_df #increasing the distance between defaulting warehouses and orders
new_ratio_df[~threshold_exceed_bool] -= ratio_df #decreasing the distance between performing warehouses and orders

print("new ratio")
print(new_ratio_df.ratio)
penalize_percent_matrix = np.reshape(np.tile(new_ratio_df.ratio.values,2), (2,10)) #shape: 2x10

print("penalize_percent_matrix")
print(penalize_percent_matrix)
distances = np.repeat(100,10).tolist() + np.repeat(200,10).tolist()
distances = np.reshape(distances, (2,10))
print(distances)
result = np.multiply(distances, penalize_percent_matrix) #(1250x10 dot product 1250x10 = shape: 1250x10)
print(result)

In [159]:
def check_feedback_of_past_k_nearest_orders(assignments):
    df = pd.DataFrame(assignments, columns=['warehouse', 'order', 'order_location', 'product', 'qty', 'distance', 'feedback'])
    predict_tuple = df.iloc[-1] #Last row (recently added assignment)
    data_df = df.iloc[:-1 , :] #Removing predict_tuple from df
    #Filtering out the previous assignments corresponding to current warehouse under consideration
    warehouse_filtered_df = data_df.loc[data_df.warehouse == predict_tuple.warehouse]
    
    #If the number of data points is less the required number of nearest neighbour then skip the warehouse penalization
    if warehouse_filtered_df.shape[0] < 5:
        return False
    
    #weights='distance' -> Close the point, Higher the influence
    knn_regressor = KNeighborsRegressor(n_neighbors=5, weights='distance')
    knn_regressor.fit(warehouse_filtered_df.order_location.values.tolist(), warehouse_filtered_df.feedback.values.tolist())
    value = knn_regressor.predict([predict_tuple.order_location])
    #if the collective feedback of the past k nearest neighbour is less than 3 then the current warehouse needs to be peanlized
    return value[0] < 3

In [160]:
def get_penalized_distances(assignments, penalize_list, distances):
    assign_df   = pd.DataFrame(assignments,   columns=['warehouse', 'order', 'order_location', 'product','qty', 'distance', 'feedback'])
    penalize_df = pd.DataFrame(penalize_list, columns=['warehouse', 'order', 'order_location', 'product','qty', 'distance', 'feedback'])
    
    if penalize_df.shape[0] == 0:
        return distances
    
    assign_group   = assign_df.groupby('warehouse')
    penalize_group = penalize_df.groupby('warehouse')
    assign_group_count   = assign_group['order'].count()
    penalize_group_count = penalize_group['order'].count()
    zero_series = pd.Series(np.repeat(0,10))

    padded_assign_group_count   = (zero_series + assign_group_count).fillna(1).tolist()    
    padded_penalize_group_count = (zero_series + penalize_group_count).fillna(0).tolist()
    ratio = np.divide(padded_penalize_group_count, padded_assign_group_count).tolist()
    threshold_exceed_bool  = pd.Series(ratio) > 0.4
    ratio_df     = pd.DataFrame(ratio, columns=['ratio'])
    new_ratio_df = pd.DataFrame(np.repeat(1.0,10), columns=['ratio'])
    new_ratio_df[threshold_exceed_bool] += ratio_df #increasing the distance between defaulting warehouses and orders
    new_ratio_df[~threshold_exceed_bool] -= ratio_df #decreasing the distance between performing warehouses and orders
    
    penalize_percent_matrix = np.reshape(np.tile(new_ratio_df.ratio.values,1250), (1250,10)) #shape: 1250x10
    result = np.multiply(distances, penalize_percent_matrix) #(1250x10 dot product 1250x10 = shape: 1250x10)
    return result

In [161]:
def assign_warehouses_to_orders(supply, warehouse_locs, demand, order_locs, enforce_feedback_for_assignment):
    """ OR-tools function to assign warehouses to orders using a max-flow min-cost solver. 
        Numbering scheme is as follows:
        warehouses = 1250 to 1259
        orders = 0 to 1249
    """
    assignments = []
    penalize_list = []
    count = 0
    distances = distance_matrix(order_locs, warehouse_locs) # Order vs Warehouse Distance (shape: 1250x10)
    start_nodes = np.repeat(np.arange(1250,1260), 1250).tolist() # eg. [1, 1, 2, 2, 3, 3]
    end_nodes   = np.tile(np.arange(0,1250), 10).tolist()        # eg. [1, 2, 3, 1, 2, 3]
    
    for i in range(400):  # iterate over products
        item_count = 0
        # Demand is being negated because they represent the sink. Quantity needs to flow from source to the sink
        supplies   = np.negative(demand[i]).tolist() + supply[i].tolist() # Concatinating demand and supplies
        capacities = np.tile(demand[i], 10).tolist()
        distances  = get_penalized_distances(assignments, penalize_list, distances)
        costs      = np.transpose(distances).ravel().astype(int).tolist() # Flattening the distance matrix 
        penalize_list = [] # Reinitializing it to empty list for accumating defaulting warehouses for the next item

        min_cost_flow = pywrapgraph.SimpleMinCostFlow() # Build solver

        for s in range(len(start_nodes)):
            min_cost_flow.AddArcWithCapacityAndUnitCost(start_nodes[s], end_nodes[s], capacities[s], costs[s])
            
        for s in range(len(supplies)):
            min_cost_flow.SetNodeSupply(s, supplies[s])

        # Optimal Warehouse for Product 'i' for 'n' Orders.
        if min_cost_flow.SolveMaxFlowWithMinCost() == min_cost_flow.OPTIMAL:
            for arc in range(min_cost_flow.NumArcs()):
                if min_cost_flow.Flow(arc) > 0:
                    warehouse = min_cost_flow.Tail(arc) - 1250
                    order = min_cost_flow.Head(arc)
                    order_location = order_loctions[order].tolist()
                    product = i
                    qty = min_cost_flow.Flow(arc)
                    cost = min_cost_flow.UnitCost(arc)
                    feedback = random.randint(1,5)
                    
                    assign = [warehouse, order, order_location, product, qty, cost, feedback]
                    assignments.append(assign)
                    item_count += qty
                    
                    if(enforce_feedback_for_assignment and check_feedback_of_past_k_nearest_orders(assignments)):
                        penalize_list.append(assign)

        count += item_count
    
    print(supply.sum(), demand.sum(), count)              
    return np.array(assignments, dtype=object), np.array(penalize_list, dtype=object)

In [162]:
%%time
assignments = assign_warehouses_to_orders(supply, warehouse_loctions, demand, order_loctions, False)[0]
feedback_enfored_assignments = assign_warehouses_to_orders(supply, warehouse_loctions, demand, order_loctions, True)[0]
assign_df   = pd.DataFrame(assignments,   columns=['warehouse', 'order', 'order_location', 'product','qty', 'distance', 'feedback'])
feedback_enfored_assignments_df = pd.DataFrame(feedback_enfored_assignments, columns=['warehouse', 'order', 'order_location', 'product','qty', 'distance', 'feedback'])

14576 9368 9368
14576 9368 9368
CPU times: user 2min 8s, sys: 1.13 s, total: 2min 9s
Wall time: 2min 9s
